<div style="background-color:#f8f9fa;
    padding:20px;
    border:2px solid #360084;
    border-radius:8px;
    margin:10px 0;">
    <div style="background-color:#fafdf0;
        line-height:2;
        text-align:center;
        border:2px solid #360084;
        padding:15px;
        border-radius:5px;">
        <div style="color:#008B8B;
            font-size:24pt;
            font-weight:700;">
            OLS Estimation Implementation Step by Step with Statsmodels
        </div>
    </div> 
    <div style="margin-top:15px; padding:10px;">
        <strong>Course:</strong> Econometrics<br>
        <strong>Topic:</strong> OLS Estimation using Python Statsmodels<br>
        <strong>Instructor:</strong> Dr. Saad Laouadi<br>
        <strong>Date:</strong> June 2025<br>
        <strong>Level:</strong> Undergraduate<br>
        <strong>Model:</strong> $SALES = \beta_0 + \beta_1 TV + \varepsilon$<br>
        <strong>Learning Objectives:</strong>
        <ul style="margin:5px 0; padding-left:20px;">
            <li>Use statsmodels for OLS estimation</li>
            <li>Extract and interpret coefficient estimates</li>
            <li>Calculate fitted values and residuals</li>
            <li>Compute goodness-of-fit measures</li>
            <li>Understand model output components</li>
        </ul>
    </div>
</div>

---

## 1. Introduction to Statsmodels

**Statsmodels** is Python's premier statistical modeling package. It provides:
- Professional econometric estimation methods
- Comprehensive model diagnostics
- Publication-ready output tables
- Extensive post-estimation analysis tools

**Why use statsmodels for OLS?**
- Handles complex data structures automatically
- Provides robust standard errors and diagnostics
- Integrates with pandas DataFrames seamlessly
- Offers multiple model specifications

---

## 2. Package Setup and Data Loading

In [ ]:
# ============================================= #
#      Environment Setup
# ============================================= #

# Import core libraries
import os
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Statsmodels components
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import diagnostic

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.precision', 4)
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

print("Statsmodels version:", sm.__version__)

print("*"*52)
print("Environment Setup".center(52))
%reload_ext watermark
%watermark -iv -v -ud -a "Dr. Saad Laouadi"
print("*"*52)

In [ ]:
# Load advertising dataset

advert = pd.read_csv("../../datasets/misc/advertising.csv")

print("Dataset Information:")
print("=" * 20)
print(f"Observations: {len(advert)}")
print(f"Variables: {advert.shape[1]}")
print("\nVariable names:")
print(advert.columns.tolist())

## 3. Explore the Data

In [ ]:
# Check the relationship between the target and other variables
sns.pairplot(data=advert, x_vars=['TV', 'Newspaper', 'Radio'], y_vars='Sales')
plt.show()

In [ ]:
# ============================================= #
#   Select the Working Variables
# ============================================= #
# Focus on our main variables
print("\nSales and TV Overview:")
print(advert[['Sales', 'TV']].describe())

In [ ]:
# Check for missing values (good practice to)
print(f"\nMissing values:")
print(f"Wage: {advert['Sales'].isnull().sum()}")
print(f"Education: {advert['TV'].isnull().sum()}")

## 4. Basic OLS Estimation with Statsmodels

### 4.1 Method 1: Using Formula API (R-style)
We will use the `ols` formula from `statsmodels` package to estimate the our model:

$$Sales = \beta_0 + \beta_1 \times TV + \varepsilon$$

In [ ]:
# Estimate model using formula interface
model_formula = smf.ols('Sales ~ TV', data=advert)
res = model_formula.fit()

# print the estimation results
print(res.summary())

---

## 4. Extracting and Understanding Coefficient Estimates

In [ ]:
print("Coefficient Estimates and Properties")
print("=" * 40)

# Extract coefficients (both methods give same results)
coefficients = res.params
intercept = coefficients['Intercept']
slope = coefficients['TV']
print("Coefficient Estimates:")
print(f"β₀ (Intercept): {intercept:.4f}")
print(f"β₁ (Education): {slope:.4f}")


In [ ]:
# Access coefficient vector
print(f"\nCoefficient vector:")
print(res.params)
print(f"Coefficient names: {res.params.index.tolist()}")

---

## 6. Fitted Values and Predictions

In [ ]:
print("Fitted Values and Predictions")
print("=" * 30)

# Extract fitted values
fitted_values = res.fittedvalues
print(f"Fitted values shape: {fitted_values.shape}")
print(f"First 10 fitted values:")
print(fitted_values.head(10))

In [ ]:
# Add the fitted values to our dataset 
advert = advert[['Sales', 'TV']]

advert['predicted_sales'] = res.fittedvalues

# Check the data
advert.head()

In [ ]:
# Manual calculation verification
manual_fitted = intercept + slope * advert['TV']
print(f"\nVerification - manual vs statsmodels fitted values match:")
print(np.allclose(fitted_values, manual_fitted))

---

## 7. Residuals Analysis

In [ ]:
print("Residual Analysis")
print("=" * 18)

# Extract residuals
residuals = res.resid
print(f"Residuals shape: {residuals.shape}")
print(f"Residual statistics:")
print(f"• Mean: {residuals.mean():.10f}")           # Should be approximately 0
print(f"• Standard deviation: {residuals.std():.4f}")
print(f"• Minimum: {residuals.min():.4f}")
print(f"• Maximum: {residuals.max():.4f}")

In [ ]:
# Verify residual properties
actual_sales = advert['Sales']
manual_residuals = actual_sales - fitted_values

print(f"\nResidual properties verification:")
print(f"• Sum of residuals: {residuals.sum():.10f}")
print(f"• Correlation with X: {np.corrcoef(advert['TV'], residuals)[0,1]:.10f}")
print(f"• Manual calculation matches: {np.allclose(residuals, manual_residuals)}")

In [ ]:
# Add the resids to the data
advert['residuals'] = res.resid
advert.head()

---

## 8. Sum of Squares Decomposition

In [ ]:
print("Sum of Squares Analysis")
print("=" * 25)

# Extract sum of squares from model
y_mean = advert['Sales'].mean()

# Calculate manually for understanding
TSS = np.sum((actual_sales - y_mean)**2)     # Total Sum of Squares
ESS = np.sum((fitted_values - y_mean)**2)    # Explained Sum of Squares
RSS = np.sum(residuals**2)                   # Residual Sum of Squares

print(f"Sum of Squares Decomposition:")
print(f"• TSS (Total): {TSS:.2f}")
print(f"• ESS (Explained): {ESS:.2f}")
print(f"• RSS (Residual): {RSS:.2f}")
print(f"• Verification TSS = ESS + RSS: {np.isclose(TSS, ESS + RSS)}")

In [ ]:
# Access from statsmodels results
model_tss = res.centered_tss  # Total sum of squares
model_ess = res.ess  # Error sum of squares (RSS in our notation)
model_mse = res.mse_resid  # Mean squared error

print(f"\nFrom statsmodels results object:")
print(f"• Centered TSS: {model_tss:.2f}")
print(f"• ESS (model's RSS): {model_ess:.2f}")
print(f"• MSE: {model_mse:.4f}")
print(f"• Root MSE: {np.sqrt(model_mse):.4f}")

In [ ]:
# Degrees of freedom
print(f"\nDegrees of freedom:")
print(f"• Total (n-1): {res.df_model + res.df_resid}")
print(f"• Model (k): {res.df_model}")  
print(f"• Residual (n-k-1): {res.df_resid}")

---

## 9. Goodness of Fit Measures

In [ ]:
print("Goodness of Fit Measures")
print("=" * 28)

# R-squared measures
r_squared = res.rsquared
adj_r_squared = res.rsquared_adj

print(f"R-squared measures:")
print(f"• R² = {r_squared:.4f}")
print(f"• Adjusted R² = {adj_r_squared:.4f}")
print(f"• R² difference = {r_squared - adj_r_squared:.6f}")

In [ ]:
# Manual calculation verification
r_squared_manual = ESS / TSS
adj_r_squared_manual = 1 - ((RSS / res.df_resid) / (TSS / (len(advert) - 1)))

print(f"\nManual calculation verification:")
print(f"• R² manual: {r_squared_manual:.4f}")
print(f"• Adj R² manual: {adj_r_squared_manual:.4f}")
print(f"• Match statsmodels: {np.allclose([r_squared, adj_r_squared], [r_squared_manual, adj_r_squared_manual])}")

In [ ]:
# Interpretation
print(f"\nInterpretation:")
print(f"• {r_squared:.1%} of wage variation explained by education")
print(f"• {(1-r_squared):.1%} of wage variation unexplained")
print(f"• Correlation coefficient: {np.sqrt(r_squared):.4f}")

# Information criteria
print(f"\nInformation Criteria:")
print(f"• AIC: {res.aic:.2f}")
print(f"• BIC: {res.bic:.2f}")
print(f"• Log-likelihood: {res.llf:.2f}")

---

## 11. Visualization of Results

In [ ]:
# ================================================ #
#  Comprehensive visualization of OLS results
# ================================================ #

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Plot 1: Scatter with regression line
ax.plot(advert['TV'], fitted_values, color='red', linewidth=2, label='OLS fit')
ax.scatter(advert['TV'], advert['Sales'], alpha=0.6, s=30, color='steelblue')
ax.set_xlabel('TV Ads')
ax.set_ylabel('Sales ($)')
ax.set_title(f'Sales vs TV Ads\n(R² = {r_squared:.3f})')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

In [ ]:
# Plot 2: Fitted vs Actual values
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(fitted_values, advert['Sales'], alpha=0.6, s=30, color='green')
ax.plot([fitted_values.min(), fitted_values.max()], 
         [fitted_values.min(), fitted_values.max()], 
         'r--', linewidth=2, label='Perfect fit line')
ax.set_xlabel('Fitted Values ($)')
ax.set_ylabel('Actual Sales ($)')
ax.set_title('Fitted vs Actual Values')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

In [ ]:
# Plot 3: Residuals vs Fitted
fig, ax3 = plt.subplots(figsize=(10, 8))
ax3.scatter(fitted_values, residuals, alpha=0.6, s=30, color='orange')
ax3.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Fitted Values ($)')
ax3.set_ylabel('Residuals ($)')
ax3.set_title('Residuals vs Fitted Values')
ax3.grid(True, alpha=0.3)

plt.show()